# Setup

In [ ]:
%reload_ext nb_black
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.graphics.gofplots import qqplot
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Function for determining Confidence Intervals
def ci_95(x1, x2):
    signal = x1.mean() - x2.mean()
    noise = np.sqrt(x1.var() / x1.size + x2.var() / x2.size)

    ci_lo = signal - 1.96 * noise
    ci_hi = signal + 1.96 * noise

    return ci_lo, ci_hi

# Exploring the data

In [ ]:
# Load in 2019 DataFrame
url = "https://raw.githubusercontent.com/WoodyBurns44/Happiness_index_Analysis/main/2019.csv"
df_2019 = pd.read_csv(url)

In [ ]:
df_2019.head()

In [ ]:
# See what data we're working with
df_2019.info()

In [ ]:
df_2019.shape

In [ ]:
df_2019.index

In [ ]:
df_2019.columns

In [ ]:
df_2019.describe()

# Is there a correlation between the GDP per capita in 2019 for the happiest and less happy countries? If so, how strong is the correlation? 

## Divide the dataset into two groups: happy countries and less happy Countries based on the median happiness score. 

In [ ]:
#Split by median to get as close to equal sized data as possible
median = df_2019["Score"].median()

In [ ]:
# Select the "Happiest" countries, above the median
happy_2019 = df_2019[df_2019["Score"] > median]

In [ ]:
happy_2019.shape

In [ ]:
#Select the less happy countries, below the median
unhappy_2019 = df_2019[df_2019["Score"] <= median]

In [ ]:
unhappy_2019.shape

In order to determine which test will most accurately find whether there is a correlation or not, I must look into the distribution of the data. First, I will test both variables for normality. 

## Test for normality 

In [ ]:
#Find the mean and medians of the data sets to see if they tell us anything abou ttheir normality
happy_2019["GDP per capita"].mean()

In [ ]:
happy_2019["GDP per capita"].median()

In [ ]:
unhappy_2019["GDP per capita"].mean()

In [ ]:
unhappy_2019["GDP per capita"].median()
# The means and medians of both samples are fairly close to eachother, which is an indication
# that the data might be normally distributed. 

In [ ]:
# Check the Kurtosis and Skewness
stats.describe(unhappy_2019["GDP per capita"])

In [ ]:
stats.describe(happy_2019["GDP per capita"])

In [ ]:
# Histogram of the GDP per capita of happy countries, with a black line showing the mean
# and an orange line showing the median.
happy_2019["GDP per capita"].hist()
plt.axvline(x=happy_2019["GDP per capita"].median(), c="orange", linestyle="solid")
plt.axvline(x=happy_2019["GDP per capita"].mean(), c="black", linestyle="solid")
plt.show()

In [ ]:
# Histogram of the GDP per capita of less happy countries, with a black line showing the mean
# and an orange line showing the median.
unhappy_2019["GDP per capita"].hist()
plt.axvline(x=unhappy_2019["GDP per capita"].median(), c="orange", linestyle="solid")
plt.axvline(x=unhappy_2019["GDP per capita"].mean(), c="black", linestyle="solid")
plt.show()

In [ ]:
## QQ plot to visualize happy countries GDP per capita relation to normal distribution
qqplot(happy_2019["GDP per capita"], line="s")
plt.show()

In [ ]:
## QQ plot to visualize less happy countries Happiness scores relation to normal distribution
qqplot(unhappy_2019["GDP per capita"], line="s")
plt.show()

In [ ]:
# Check normality with a Violin plot
sns.violinplot(x="GDP per capita", data=unhappy_2019, color="orange")
sns.violinplot(x="GDP per capita", data=happy_2019)
plt.show()

In [ ]:
j, p = stats.jarque_bera(unhappy_2019["GDP per capita"])

In [ ]:
j

In [ ]:
p < 0.05

The result of the Jarque-Bera test indicates that the distribution is not perectly normal. However, since the sample size is small and all of the other tests indicate normality, I will treat it as normal. 

Since both happy and less happy countries GDP per capita appear to be normal, I will perform a Students T-test to determine if there is variance between the groups. 

## Student T-Test 

The Students T-test is used to detect if the means are different between two groups. 
* $H_o$ : Both developing and developed countries have the same mean of GDP per capita
* $H_a$ : The mean of GDP per capita differs between developing and developed countries 

In [ ]:
ttest_score, ttest_p = stats.ttest_ind(
    happy_2019["GDP per capita"], unhappy_2019["GDP per capita"]
)

In [ ]:
ttest_score, ttest_p

In [ ]:
ttest_p < 0.05

The Students  T-test indicates that the null hypothesis can be rejected and that the distribution of GDP per capita differs between happy and less happy countries. 

## How significant is the difference in GDP per capita between the happiest and less happy countries?

In order to determine the difference in means between happy and less happy countries' GDP per Capital I will calculate a confidence interval and then bootstrap to test that calculation.

In [ ]:
# Calculating low and high confidence intervals using function defined above
ci_95(happy_2019["GDP per capita"], unhappy_2019["GDP per capita"])

I can say with 95% confidence that there is a .45 and .63 difference in the GDP per Capita of happy and less happy countries, in favor of the more happy countries.  

In [ ]:
# Testing confidence interval with a bootstrap

mean_diffs = []
for i in range(10000):
    control_sample = happy_2019["GDP per capita"].sample(frac=1.0, replace=True)
    treatment_sample = unhappy_2019["GDP per capita"].sample(frac=1.0, replace=True)
    mean_diff = control_sample.mean() - treatment_sample.mean()
    mean_diffs.append(mean_diff)

In [ ]:
low_ci = np.percentile(mean_diffs, 2.5)
high_ci = np.percentile(mean_diffs, 97.5)

In [ ]:
low_ci, high_ci

The sample bootstrapping confirms my calculations that there is, with 95% certainty, a .45 and .63 difference in the GDP per Capita of happy and less happy countries, in favor of the more happy countries.

# Has the happiness of the world changed from 2015 to 2019? If so, in what way and how much? 

## Investigate the DataFrame for 2015 data

In [ ]:
# Use Pandas to import the 2015 Dataframe
url_2015 = "https://raw.githubusercontent.com/WoodyBurns44/Happiness_index_Analysis/main/2015.csv"
df_2015 = pd.read_csv(url_2015)

In [ ]:
df_2015.head()

In [ ]:
df_2015.info()

In [ ]:
df_2015.describe()

In [ ]:
df_2015.columns

## Isolate the Happiness columns for 2015 and 2019

In [ ]:
# Create Dataframes for the 2015 and 2019 data
happy15 = df_2015["Happiness Score"]
# happy15.head()
happy15.index

In [ ]:
happy19 = df_2019["Score"]
# happy19.head()
happy19.index

## Test for normality of happiness in 2015 and 2019

### 2015

In [ ]:
happy15.mean()

In [ ]:
happy15.hist()
plt.axvline(x=happy15.mean(), c="orange")
plt.axvline(x=happy15.median(), c="black")
plt.show()

In [ ]:
qqplot(happy15, line="s")
plt.show()

Use the Jarque-Bera test for normality
* $H_o$ : The data comes from a normally distributed set. 
* $H_a$ : The data does not come form a normally distributed set. 

In [ ]:
j15, p15 = stats.jarque_bera(happy15)

In [ ]:
j15

In [ ]:
p15

In [ ]:
p < 0.05

Since we cannot reject the null, it can be assumed that the data comes from a normal distribution. 

### 2019

In [ ]:
happy19.mean()

In [ ]:
happy19.hist()
plt.axvline(x=happy19.mean(), c="orange")
plt.axvline(x=happy19.median(), c="black")
plt.show()

In [ ]:
qqplot(happy19, line="s")
plt.show()

In [ ]:
j19, p19 = stats.jarque_bera(happy19)

In [ ]:
j19

In [ ]:
p19

Since we cannot reject the null, it can be assumed that the data comes from a normal distribution.

## Perform an independent T-test

In [ ]:
t_15_to_19, p_15_to_19 = stats.ttest_ind(happy15, happy19)

In [ ]:
t_15_to_19

In [ ]:
p_15_to_19

## Result

There does not appear to be a significant diffance in the overall happiness of the world in 2019 as compared with 2015. 

# Which factors are most strongly correlated to the overall happiness score in 2019?

Make a Spearman Correlatoin matrix to test for correlations between all numeric categories

In [ ]:
spearman_correlations = df_2019.corr(method="spearman")
spearman_correlations

Translate matrix into a heatmap for better visualization of the correlations. 

* code for heatmap inspired by Jesper Dramsch on Kaggle (https://www.kaggle.com/jesperdramsch/the-reason-we-re-happy). 

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(24, 8))
sns.heatmap(
    spearman_correlations,
    vmin=-1,
    vmax=1,
    ax=ax[0],
    center=0,
    cmap="viridis",
    annot=True,
)
sns.heatmap(
    spearman_correlations,
    vmin=-0.25,
    vmax=1,
    ax=ax[1],
    center=0,
    cmap="Accent",
    annot=True,
    linecolor="white",
)

The heat map gives us a lot of information, including:
* Validates that there is a strong correlation between GDP per capita and happiness score
* Shows that there is very strong correlation between the following fields and the happiness score:
    * Social Support
    * Healthy Life Expectancy 
    * Freedom to make Life choices

# Appendix: Further Exploration 

In [ ]:
url_2016 = "https://raw.githubusercontent.com/WoodyBurns44/Happiness_index_Analysis/main/2016.csv"
url_2017 = "https://raw.githubusercontent.com/WoodyBurns44/Happiness_index_Analysis/main/2017.csv"
url_2018 = "https://raw.githubusercontent.com/WoodyBurns44/Happiness_index_Analysis/main/2018.csv"

In [ ]:
df_2016 = pd.read_csv(url_2016)
df_2017 = pd.read_csv(url_2017)
df_2018 = pd.read_csv(url_2018)

In [ ]:
df_2016.head(1)

In [ ]:
df_2015.head(1)

In [ ]:
merge_columns = [
    "Country",
    "GDP",
    "Family",
    "Life",
    "Freedom",
    "Generosity",
    "Trust",
]


def merge_fun(year_df, year):
    df = pd.DataFrame()
    add_columns = []
    for i in merge_columns:
        add_columns.extend(x for x in year_df.columns if i in x)
    df[merge_columns] = year_df[add_columns]
    df["Happiness Score"] = year_df[[x for x in year_df.columns if "Score" in x]]
    df["Year"] = year
    df = df.set_index(["Country", "Year"])
    return df

In [ ]:
df = merge_fun(df_2015, 2015)

In [ ]:
df = df.append(merge_fun(df_2016, 2016), sort=False)

In [ ]:
df = df.append(merge_fun(df_2017, 2017), sort=False)

In [ ]:
df_2018 = df_2018.rename(
    columns={
        "Healthy life expectancy": "Life",
        "Perceptions of corruption": "Trust",
        "Social support": "Family",
    },
)

In [ ]:
df = df.append(merge_fun(df_2018, 2018), sort=False)

In [ ]:
df_2019 = df_2019.rename(
    columns={
        "Social support": "Family",
        "Healthy life expectancy": "Life",
        "Perceptions of corruption": "Trust",
    }
)

In [ ]:
df = df.append(merge_fun(df_2019, 2019), sort=False)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.index

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={"Happiness Score": "Happiness_Score"})

In [ ]:
df_test = df.reset_index()

In [ ]:
df_test.head()

In [ ]:
df_test["Year"].describe()

In [ ]:
sns.set(rc={"figure.figsize": (11.7, 8.27)})
happy_plot = sns.lineplot(
    x="Year",
    y="Happiness_Score",
    hue="Country",
    legend="brief",
    data=df_test,
)
# happy_plot.legend(loc=10)

In [ ]:
df_test.groupby("Year").agg("count")